In [ ]:
#recover HBNet positions to use for precision recall curve 

import pandas as pd
import sys,os,json
import tempfile
import numpy as np
import argparse
from pyrosetta import *
import pyrosetta.rosetta.core.scoring as scoring
from pyrosetta.rosetta.protocols.minimization_packing import MinMover
from pyrosetta.rosetta.protocols import hbnet
import time
from pyrosetta.toolbox import cleanATOM
import seaborn as sns
import matplotlib.pyplot as plt
from pyrosetta.rosetta.core.select.residue_selector import InterGroupInterfaceByVectorSelector, ChainSelector, NotResidueSelector
from pyrosetta.rosetta.protocols.hbnet import HBNetStapleInterface


In [ ]:
pyrosetta.init()

In [ ]:
def setupHBNet(scorefxn):
    #returns hbnet to apply to every pose to extract existing Zibo networks
    setup_using_string = True
    if setup_using_string:
        hbnet = pyrosetta.rosetta.protocols.rosetta_scripts.XmlObjects.create_from_string("""
            <MOVERS>
            <HBNetStapleInterface name="hbnet" monte_carlo="true" min_network_size="4" store_network_scores_in_pose="true" min_helices_contacted_by_network="4" find_only_native_networks="True" />
            </MOVERS>
            """).get_mover("hbnet")
    else:
        hbnet = HBNetStapleInterface()
        hbnet.set_monte_carlo_branch(True)
    hbnet.set_total_num_mc_runs(1)
    hbnet.set_score_function(scorefxn)
    return hbnet

def setupHBNetRelaxedRecovery(scorefxn):
    #using a relaxed criteria to recover HbNet positions 
    #(as dimer design process can apparently distort and break the networks of size 4)
    setup_using_string = True
    if setup_using_string:
        hbnet = pyrosetta.rosetta.protocols.rosetta_scripts.XmlObjects.create_from_string("""
            <MOVERS>
            <HBNetStapleInterface name="hbnet" monte_carlo="true" min_network_size="3" store_network_scores_in_pose="true" min_helices_contacted_by_network="3" hb_threshold="-0.3" find_only_native_networks="True" />
            </MOVERS>
            """).get_mover("hbnet")
    else:
        hbnet = HBNetStapleInterface()
        hbnet.set_monte_carlo_branch(True)
    hbnet.set_total_num_mc_runs(10)
    hbnet.set_score_function(scorefxn)
    return hbnet

def extractHBNetsFromOrigStructure(pose, scorefxn, hbnet):
    start_pose = Pose()
    start_pose.assign(pose)
    pose.assign(start_pose)
    hbnet.apply(pose)
    dummy = hbnet.get_native_vec() #vector of native Hbnet networks in the structure
    networksRes = [] #storing things as tuples of chain:residues
    networks = 0
    while dummy:
        x = dummy.pop()
        #print ("total residues in network: ", len(x.residues))
        actualResidues = x.residues # a vector of HBondResStructs
        actualRes = []
        networks += 1
        #print(x.outstring)
        while actualResidues: #save as chain:resnum
            resCurrent = actualResidues.pop()
            networksRes.append(resCurrent.chainid + ":" + str(resCurrent.resnum))
        #networks.append(actualRes)
    return networks, networksRes

In [ ]:
#functions to find hbnet info and add to csv file

def attachHBNetInfo(csvName, csvOutName):
    #takes in csv of pdb names for a folder containing the designed dimer pdbs 
    #calculates number of networks & positions of networks in chains and saves a new csv of the information 
    csvFile = pd.read_csv(csvName)
    dirWithPdbs = "./test_set_structures/"
    scorefxn = get_fa_scorefxn()
    csvFile['HBNetsRecovered'] = None
    csvFile['RecoveredHBNetPositions'] = None
    hbNetToUse = setupHBNet(scorefxn)  # much slower, but maybe won't crash on 156
    for ind, row in csvFile.iterrows():
        print ("ON: ", ind, " OUT OF ", csvFile.shape[0])
        name = row['structure']
        if 'negative' in name:
            name = dirWithPdbs + "trRosettaPredicted_" + name + ".pdb"
        else:
            name = dirWithPdbs + name + ".pdb"
        cleanATOM(name)
        cleanPDBName = name[:-4] + ".clean.pdb"
        pose = pose_from_pdb(cleanPDBName)
        networks, residues = extractHBNetsFromOrigStructure(pose, scorefxn, hbNetToUse)
        csvFile.at[ind, 'HBNetsRecovered'] = networks
        csvFile.at[ind, 'RecoveredHBNetPositions'] = "|".join(residues)  # all residues in a HBNet network
    print(csvFile.shape)
    print(csvFile[csvFile['HBNetsRecovered'] >= 2].shape)
    csvFile.to_csv(csvOutName)


def attachHBNetInfoRelaxedQualifications(csvName, csvOutName):
    # csvName = "with_energy_termsdimerStructuresToScore_notposOnly.csv"
    csvFile = pd.read_csv(csvName)
    dirWithPdbs = "./test_set_structures/"
    scorefxn = get_fa_scorefxn()
    csvFile['HBNetsRecovered'] = None
    csvFile['RecoveredHBNetPositions'] = None
    hbNetToUse = setupHBNetRelaxedRecovery(scorefxn) 
    for ind, row in csvFile.iterrows():
        print("ON: ", ind, " OUT OF ", csvFile.shape[0])
        name = row['structure']
        if 'negative' in name:
            name = dirWithPdbs + "trRosettaPredicted_" + name + ".pdb"
        else:
            name = dirWithPdbs + name + ".pdb"
        cleanATOM(name)
        cleanPDBName = name[:-4] + ".clean.pdb"
        pose = pose_from_pdb(cleanPDBName)
        networks, residues = extractHBNetsFromOrigStructure(pose, scorefxn, hbNetToUse)
        # print ("found: ", len(networks))
        csvFile.at[ind, 'HBNetsRecovered'] = networks
        csvFile.at[ind, 'RecoveredHBNetPositions'] = "|".join(residues)  # all residues in a HBNet network
    print(csvFile.shape)
    print(csvFile[csvFile['HBNetsRecovered'] >= 2].shape)
    csvFile.to_csv(csvOutName)

In [ ]:
def attachHBNetInfoAs0And1s(csvName, saveNameNumpy1, saveNameNumpy2, saveNameCSV):
    #takes csv that has HbNet info attahced to it already, saves numpy arrays with 1's in hbnet positions
    #0's in other positions for doing PR curve
    csvFile = pd.read_csv(csvName)
    #filter out -1 values
    csvFile = csvFile[~(csvFile['HBNetsRecovered'] == -1)]
    print (csvFile.shape)
    overallMatrixA = []
    overallMatrixB = []
    for ind, row in csvFile.iterrows():
        print("ON: ", ind, " OUT OF ", csvFile.shape[0])
        name = row['structure']
        networksRecovered = row['HBNetsRecovered']
        if networksRecovered > 0:
            dummyRowA = [0] * row['lenA']
            dummyRowB = [0] * row['lenB']
            positions = row['RecoveredHBNetPositions'].split("|")
            posA = [int(x.split(":")[1]) - 1 for x in positions if 'A' in x]
            for pos in posA:
                dummyRowA[pos] = 1
            overallMatrixA.append(dummyRowA)
            posB = [int(x.split(":")[1]) - (1 + row['lenA']) for x in positions if 'B' in x]
            for pos in posB:
                dummyRowB[pos] = 1
            overallMatrixB.append(dummyRowB)
            #make numpy arrays
        else:
            dummyRowA = [0] * row['lenA']
            dummyRowB = [0] * row['lenB']
            overallMatrixA.append(dummyRowA)
            overallMatrixB.append(dummyRowB)
    print ("rows a: ", len(overallMatrixA))
    print ("rows b: ", len(overallMatrixB))

    #to numpy
    numpyA = np.asarray(overallMatrixA)
    print(numpyA.shape)
    np.save(saveNameNumpy1, numpyA)

    numpyB = np.asarray(overallMatrixB)
    np.save(saveNameNumpy2, numpyB)
    print(numpyB.shape)
    csvFile.to_csv(saveNameCSV, index_label=False)

In [ ]:
#running on test set 
newName = "binder_names.csv"
attachHBNetInfoAs0And1s(newName, "HBNet_positions_updated_1.npy", "HBNet_positions_updated_2.npy", "hbnet_positions.csv")
